# Análisis automático orientado a corrección de textos

En este cuaderno estoy probando diferentes **dimensiones de corrección y análisis de textos** mediante el uso de modelos de lenguaje (Azure OpenAI).  

El objetivo es evaluar y detectar errores en tres grandes dimensiones:

1. **Ortografía**  
   - Acentual: reglas ortográficas que rigen la acentuación de la lengua española.
   - Literal: omisiones, sustituciones, adiciones de letras, mayúsculas.  
   - Puntual: uso incorrecto de comas, puntos, signos de interrogación y exclamación, etc. 

2. **Vocabulario**  
   - Amplitud: detección de repeticiones innecesarias dentro de un mismo párrafo, considerando sustantivos, adjetivos, verbos y adverbios.  
   - Precisión léxica: reconocimiento del uso incorrecto del significado de palabras, de los extranjerismos incorrectamento formulados y de palabras e ideas reiteradadas excesivamente.
   - Adecuación al registro de habla formal: identificación del empleo de modo de comunicación escrita que se ajusta a la norma culta de la lengua, caracterizado por un uso cuidadoso y preciso del vocabulario y la gramática.

3. **Cohesión textual**  
   - Concordancia gramatical: sujeto-predicado, sustantivo-adjetivo, tiempos verbales en enumeraciones y cláusulas condicionales.  
   - Conexión textual: uso de preposiciones, locuciones, conectores adecuados y organizadores textuales.  
   - Referencias y correferencias: detección de pronombres o expresiones sin referente claro, como *esto*, *lo anterior*, etc.  

Cada dimensión se prueba sobre los textos del dataset **`escritos_seleccionados_a1.xlsx`**, y los resultados se almacenan en archivos JSON, manteniendo un registro estructurado de los errores detectados.  

El flujo de trabajo consiste en evaluar cada subdimensión mediante llamadas a la API de Azure OpenAI, siguiendo los criterios definidos en la rúbrica elaborada por expertos (versión del 15 de septiembre de 2025), la cual establece tanto los aspectos de evaluación como los puntajes asociados. Para cada una de las subdimensiones se realiza una llamada independiente por cada uno de los 10 textos contenidos en el documento escritos_seleccionados_ort.xlsx.

*Documento elaborado por Renata Ramírez, practicante MIDEUC.*

## 1. Ortografía

### Acentual

In [20]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_acentual = """
Eres un asistente experto en corrección de textos.

EVALÚA Y CORRIGE EXCLUSIVAMENTE los errores en el texto relacionados con el uso de tildes en español. 
Considera las reglas generales (palabras agudas, graves, esdrújulas y sobreesdrújulas) 
y las reglas especiales (acento diacrítico y acento dierético).

Tu respuesta debe seguir exactamente esta estructura:

{
  "errores_acentuales": {
        "error_1": ["error → corrección"],
        "error_2": ["error → corrección"],
        "error_3": ["error → corrección"],
        ...
  }
}

INSTRUCCIONES CRÍTICAS:
- SOLO reporta errores en los que la diferencia entre error y corrección sea la presencia o ausencia de una tilde.
- NO señales problemas de ortografía, puntuación, vocabulario ni otros aspectos.
- NO incluyas el texto corregido completo.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_acentual},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_acentual.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\ort_acentual.json


### Literal

In [21]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_literal = """
Eres un asistente experto en corrección de textos. 

Corrige ÚNICAMENTE los siguientes tipos de errores:

1. Mayúsculas ausentes: inicio de párrafo, tras punto, nombres propios. Ej: ". ahora" → ". Ahora", "madrid" → "Madrid".
2. Omisión de letras: ej. "alchol" → "alcohol", "gerra" → "guerra", "aora" → "ahora".
3. Sustitución de letras: ej. "girafa" → "jirafa", "obeja" → "oveja".
4. Adición de letras: ej. "rrallado" → "rallado", "exhuberante" → "exuberante". Incluye "en carro": ej. “apartir” → “a partir”, “através” → “a través”.

INSTRUCCIÓN CRÍTICA:
- NO revises ni corrijas tildes (acentos ortográficos). 
- Cualquier palabra cuya única diferencia con la forma correcta sea la presencia o ausencia de una tilde DEBE SER IGNORADA.
- Si la única “corrección” que encuentras sería añadir o quitar una tilde, NO la reportes.

Formato de respuesta OBLIGATORIO (solo JSON válido):

{
  "errores_ortografia_literal": {
        "error_1": ["error → corrección"],
        "error_2": ["error → corrección"],
        ...
  }
}

Reglas:
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json.
- El JSON debe empezar con { y terminar con }.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_literal},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_literal.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\ort_literal.json


### Puntual

In [22]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_puntual = """
Eres un asistente experto en corrección de textos. 

Tu tarea es corregir ÚNICAMENTE errores relacionados con signos de puntuación, de acuerdo con las normas de la RAE:

- Uso de la coma.
- Uso del punto (seguido, aparte y final).
- Uso del punto y coma.
- Uso de signos de exclamación e interrogación.

INSTRUCCIONES CRÍTICAS:
- NO revises ni corrijas tildes (acentos ortográficos). 
- NO corrijas errores de ortografía literal (letras omitidas, añadidas, sustituidas, mayúsculas, etc.).
- NO evalúes vocabulario ni estilo.
- Si un posible “error” solo se diferencia de la forma correcta por la presencia o ausencia de una tilde, DEBES IGNORARLO.

Formato de salida (solo JSON válido):

{
  "errores_ortografia_puntual": {
        "error_1": ["error → corrección"],
        "error_2": ["error → corrección"],
        ...
  }
}

Reglas:
- Cada error y corrección debe mostrarse con el fragmento MÁS BREVE posible que contenga el error (no más de 20 caracteres).
- NO incluyas el texto completo corregido.
- Devuelve SOLO JSON válido, sin markdown ni ```json.
- El JSON debe comenzar con { y terminar con }.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_puntual},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_puntual.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\ort_puntual.json


## 2. Vocabulario

### Amplitud

In [24]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_amplitud = """
Eres un asistente experto en corrección de textos.

Analiza EXCLUSIVAMENTE la dimensión de AMPLITUD DE VOCABULARIO en un único PÁRRAFO recibido, según las siguientes reglas:

- El emisor debe usar sinónimos y correferencias para evitar repeticiones innecesarias.
- Se sanciona la repetición de una misma palabra dentro de un mismo párrafo.
- EXCEPCIÓN: No sancionar repeticiones cuando se trate de una figura retórica de énfasis.
- Solo contabilizar repeticiones en SUSTANTIVOS, ADJETIVOS, VERBOS y ADVERBIOS.
- No contar conjugaciones verbales ni perífrasis verbales (auxiliar + infinitivo/gerundio/participio).
- Se analiza únicamente el párrafo entregado.

Tu respuesta debe seguir esta estructura:

{
  "errores_vocabulario_amplitud": {
        "palabra1": ["n veces"],
        "palabra2": ["n veces"]
  }
}

INSTRUCCIONES PARA EL FORMATO:
- Devuelve SOLO palabras repetidas con su número de apariciones.
- NO incluyas texto corregido ni el párrafo original.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    parrafos = [p.strip() for p in texto.split("\n") if p.strip()]  

    errores_por_texto = {"id_texto": i, "errores_vocabulario_amplitud": {}}

    for j, parrafo in enumerate(parrafos, start=1):
        messages = [
            {"role": "system", "content": prompt_amplitud},
            {"role": "user", "content": parrafo}
        ]

        try:
            response = client.chat.completions.create(
                model=DEPLOYMENT_GPT,
                messages=messages,
                # temperature=0,
                response_format={"type": "json_object"}
            )

            respuesta_limpia = response.choices[0].message.content.strip()
            resultado_json = json.loads(respuesta_limpia)

            errores_por_texto["errores_vocabulario_amplitud"][f"parrafo_{j}"] = resultado_json.get("errores_vocabulario_amplitud", {})

        except json.JSONDecodeError as e:
            print(f"Error al parsear JSON en texto {i}, párrafo {j}: {e}")
            print("Respuesta cruda recibida:")
            print(respuesta_limpia)

    resultados.append(errores_por_texto)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_amplitud.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\voc_amplitud.json


### Precisión léxica

In [25]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_precision = """
Eres un asistente experto en corrección de textos.

Analiza EXCLUSIVAMENTE la dimensión de PRECISIÓN LÉXICA, según los siguientes criterios:

A) USO INCORRECTO DE SIGNIFICADOS:
- Detecta palabras usadas con un significado equivocado que cambien el sentido del texto.
- Ejemplo: “absceso” por “acceso”, “actitud” por “aptitud”.

B) EXTRANJERISMOS:
- Señala extranjerismos que NO estén en comillas, cursiva o introducidos explícitamente por el autor.
- Ejemplo incorrecto: usar "online" sin comillas ni contexto en vez de “en línea”.
- No sanciones extranjerismos correctos o presentados como citas.

C) REDUNDANCIAS:
- Detecta repeticiones innecesarias de palabras o ideas.
- Ejemplo: “salir para afuera”, “volar por los aires”, “hace un tiempo atrás”.

Tu respuesta debe seguir esta estructura:

{
  "errores_precision": {
        "error_1": ["error → corrección", CRITERIO],
        "error_2": ["error → corrección", CRITERIO],
        ...,
        "error_n": ["error → corrección", CRITERIO]
  }
}

REGLAS:
- En CRITERIO indica "USO INCORRECTO DE SIGNIFICADOS", "EXTRANJERISMO" o "REDUNDANCIA".
- Enumera todos los errores encontrados bajo la clave "errores_vocabulario_precision".
- En cada caso, muestra el fragmento MÁS BREVE posible con el error.
- Da una corrección propuesta o explica por qué es impreciso.
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_precision},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_precision.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\voc_precision.json


### Registro formal

In [26]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_formal = """
Eres un asistente experto en corrección de textos.

Analiza EXCLUSIVAMENTE la dimensión de VOCABULARIO DE ADECUACIÓN AL REGISTRO DE HABLA FORMAL, según los siguientes criterios:

1) SEGUNDA PERSONA SINGULAR:
- Detecta uso del pronombre “tú” o verbos conjugados en segunda persona singular.
- Ejemplo: "Cuando postulas a un trabajo, tú tienes que pasar una entrevista".

2) ABREVIACIONES:
- Señala formas abreviadas propias de la oralidad.
- Ejemplo: “profe” por “profesor/a”, “finde” por “fin de semana”.

3) COLOQUIALISMOS: uso de expresiones coloquiales (expresiones propias de la conversación informal); por ejemplo: “socio” por “amigo”, “cachar” por “comprender” o “darse cuenta”, “pega” por “trabajo”; el uso de calificativos más expresivos: “mina” por “guapa”, “bacán” por “excelente”, “pichiruche” por “insignificante”; el uso de diminutivos y el prefijo “super” para amplificar valor.

4) ESCRITURA DE PALABRAS CON SÍMBOLOS O CARACTERES NO GRAFEMATICOS: sustitución de letras por símbolos para representar palabras (ejemplo: "q’" por "que", "total%" por "totalmente", "publicaØ" por "publicación").

Tu respuesta debe seguir esta estructura:

{
  "errores_vocabulario_formal": {
        "error_1": ["error → corrección sugerida", CRITERIO],
        "error_2": ["error → corrección sugerida"], CRITERIO],
        ...,
        "error_n": ["error → corrección sugerida", CRITERIO]
  }
}

REGLAS:
- En "CRITERIO", indica si el criterio es "SEGUNDA PERSONA SINGULAR", "ABREVIACIONES", "COLOQUIALISMOS" o "CÓDIGO ESCRITO NO FORMAL".
- Enumera todos los errores encontrados bajo la clave "errores_vocabulario_formal".
- En cada caso, muestra el fragmento MÁS BREVE posible con el error.
- Ofrece una corrección sugerida acorde al registro formal.
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- NO INCLUYAS ERRORES DE ORTOGRFÍA ACENTUAL LITERAL NI PUNTUAL.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_formal},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_formal.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

# 4) CÓDIGO ESCRITO NO FORMAL:
# - Detecta usos innecesarios de caracteres o símbolos en el español, sin incluiir til
# que alteren la escritura formal con símbolos o caracteres que no son letras ni tildes.
# - Ejemplos: "q’" por "que", "total%" por "totalmente", "publicaØ" por "publicación".

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\voc_formal.json


## 3. Cohesión

### Concordancia gramatical

In [27]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_concordancia = """
Eres un asistente experto en corrección de textos.  

Analiza EXCLUSIVAMENTE la dimensión de **COHESIÓN TEXTUAL: CONCORDANCIA GRAMATICAL**, según estas reglas:  

Se consideran errores de concordancia:
- Falta de concordancia entre **sustantivo y adjetivo**.  
- Falta de concordancia entre **sujeto y predicado**.  
- Falta de concordancia entre **pronombres relativos y sus antecedentes**.  
- Falta de consistencia en el uso de **tiempos verbales**, SOLO en los siguientes casos:  
  A) Cambio de tiempo verbal en una enumeración de acciones.  
  B) Uso de tiempos no adecuados en cláusulas condicionales y concesivas.  

Tu respuesta debe seguir esta estructura:  

{
  "errores_cohesion_concordancia": {
    "error_1": ["error → corrección", "tipo: sustantivo-adjetivo"],
    "error_2": ["error → corrección", "tipo: sujeto-predicado"],
    "error_3": ["error → corrección", "tipo: pronombre-relativo"],
    "error_4": ["error → corrección", "tipo: A"],
    "error_5": ["error → corrección", "tipo: B"],
    ...
    "error_n": ["error → corrección", "tipo: ..."]
  }
}

REGLAS:  
- En "errores_cohesion_concordancia" lista todos los errores detectados.  
- Cada error debe mostrarse con el fragmento MÁS BREVE posible (no frases completas).  
- Especifica en cada caso el tipo de error:  
  - "sustantivo-adjetivo"  
  - "sujeto-predicado"  
  - "pronombre-relativo"  
  - "A" (enumeración de acciones)  
  - "B" (cláusulas condicionales o concesivas)  
- NO incluyas el texto completo corregido.  
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.  
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_concordancia},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_concordancia.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\coh_concordancia.json


### Conexión textual

In [28]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_conexion = """
Eres un asistente experto en corrección de textos.  

Analiza EXCLUSIVAMENTE la dimensión de **COHESIÓN TEXTUAL: CONEXIÓN TEXTUAL**, según estas reglas:  

Se consideran errores de conexión textual:  
1. Uso inadecuado de preposiciones o rección: falta la preposición o se usa una preposición incorrecta.  
   - Incluye queísmo (uso de "que" sin "de" cuando corresponde) y dequeísmo (uso de "de que" cuando solo corresponde "que").  
2. Uso inadecuado de locuciones preposicionales u otras construcciones lexicales de carácter conectivo (ej.: "sobre la base de", "a razón de", "en relación con") **cuando afecten la lógica de conexión oracional y/o textual**.  
3. Ausencia de conectores o uso de conectores inadecuados para expresar la idea que se busca; mal uso de organizadores textuales (ej.: "en primer lugar", "por un lado", etc.).  

Tu respuesta debe seguir esta estructura:  

{
  "errores_cohesion_conexion": {
    "error_1": ["error → corrección", "tipo: preposición/rección"],
    "error_2": ["error → corrección", "tipo: queísmo"],
    "error_3": ["error → corrección", "tipo: dequeísmo"],
    "error_4": ["error → corrección", "tipo: locución-preposicional"],
    "error_5": ["error → corrección", "tipo: conector-ausente"],
    "error_6": ["error → corrección", "tipo: conector-inadecuado"],
    ...
    "error_n": ["error → corrección", "tipo: ..."]
  }
}

REGLAS:  
- En "errores_cohesion_conexion" lista todos los errores detectados.  
- Cada error debe mostrarse con el fragmento MÁS BREVE posible (no frases completas).  
- Especifica siempre el tipo de error con una de estas categorías:  
  - "preposición/rección"  
  - "queísmo"  
  - "dequeísmo"  
  - "locución-preposicional"  
  - "conector-ausente"  
  - "conector-inadecuado"  
- NO incluyas el texto completo corregido.  
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.  
- NO INCLUYAS ERRORES DE ORTOGRFÍA ACENTUAL LITERAL NI PUNTUAL.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_conexion},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_conexion.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\coh_conexion.json


### Referencias y correferencias

In [29]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_referencias = """
Eres un asistente experto en corrección de textos.  

Analiza EXCLUSIVAMENTE la dimensión de **COHESIÓN TEXTUAL: REFERENCIAS Y CORREFERENCIAS**, según estas reglas:  

Se consideran errores de referencia y correferencia:  
- Pronombres o mecanismos de correferencia que no tengan un referente claro en el texto (ej.: "esto", "lo anterior", "esa situación").  

Tu respuesta debe seguir esta estructura:  

{
  "errores_cohesion_referencia": {
    "error_1": ["error → corrección", "tipo: pronombre-sin-referencia"],
    "error_2": ["error → corrección", "tipo: pronombre-sin-referencia"],
    ...
    "error_n": ["error → corrección", "tipo: pronombre-sin-referencia"]
  }
}

REGLAS:  
- En "errores_cohesion_referencia" lista todos los errores detectados.  
- Cada error debe mostrarse con el fragmento MÁS BREVE posible que contenga el error (no frases completas).  
- Todos los errores se clasifican como "tipo: pronombre-sin-referencia".  
- NO incluyas el texto completo corregido.  
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON. 
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_referencias},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_referencias.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\coh_referencias.json
